In [2]:
!curl https://raw.githubusercontent.com/aamritpa/CMPT-Data-Covid-19/master/finalDataPart1.csv --output ./finalDataPart1.csv
!curl https://raw.githubusercontent.com/aamritpa/CMPT-Data-Covid-19/master/finalDataPart2.csv --output ./finalDataPart2.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 57.6M  100 57.6M    0     0  31.7M      0  0:00:01  0:00:01 --:--:-- 31.6M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 57.0M  100 57.0M    0     0  35.3M      0  0:00:01  0:00:01 --:--:-- 35.3M


In [10]:
# Importing files
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score
from sklearn.metrics import accuracy_score
import datetime
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import classification_report

In [11]:
def to_timestamp(d):
    return d.timestamp()


def makeDatetime(data):
    return datetime.datetime.strptime(data, '%Y-%m-%d')
    
def DataImportAndProcessing():
    # Importing Data
    data1 = pd.read_csv('finalDataPart1.csv')
    data2 = pd.read_csv('finalDataPart2.csv')
    # Appending IF NEEDED
    data = data1.append(data2)
    data=data.reset_index(drop=True)
    data = data.drop(columns=['index'])
    data['date_confirmation'] = data['date_confirmation'].apply(makeDatetime)
    data['date_confirmation'] = data['date_confirmation'].apply(to_timestamp) 
    # Encoding Label i.e outcome column
    data['outcome'].replace({'nonhospitalized':0, 'recovered':1, 'hospitalized':2, 'deceased':3},inplace=True)
    # Introducing dummies into data to get rid of all the categorical data
    encodedCountry = pd.get_dummies(data['country'])
    encodedProvince = pd.get_dummies(data['province'])
    encodedSex = pd.get_dummies(data['sex'])
    data = pd.concat([data, encodedSex, encodedProvince,encodedCountry], axis=1)
    print("length of filled data: ", len(data))
    data = data.drop(columns=['province','country','source', 'additional_information', 'sex'])
    return data

scoring = {"recall":make_scorer(recall_score, average = 'macro'),
           "recall_Deceased":  make_scorer(recall_score, labels = [3],  average= None),
           "Accuracy": make_scorer(accuracy_score)}


def GridXGB(X, y, mcw):
    grid = {"max_depth": [10, 20, 30]}
    
    clf = GridSearchCV(XGBClassifier(min_child_weight = mcw),
                  param_grid=grid,
                  scoring=scoring, refit=False, return_train_score=True, cv = 5, verbose = 2, n_jobs=-1)
    
    clf.fit(X, y)
    results = clf.cv_results_
    pd.DataFrame(results).to_csv(f"XGB_{mcw}.csv")
    display(pd.DataFrame(results))


def GridRF(X, y, md):
    grid = {"n_estimators": [50, 75, 100, 150, 200]}
    
    clf = GridSearchCV(RandomForestClassifier(max_depth = md),
                  param_grid=grid,
                  scoring=scoring, refit=False, return_train_score=True, cv =5, verbose = 2, n_jobs=-1)
    
    clf.fit(X, y)

    results = clf.cv_results_
    pd.DataFrame(results).to_csv(f"RF_{md}.csv")
    display(pd.DataFrame(results))



def GridKNN(X, y, w):
    grid = {"n_neighbors": [2, 3, 5, 7, 9, 11, 13, 20]}
    clf = GridSearchCV(KNeighborsClassifier(weights = w ),
                  param_grid=grid,
                  scoring=scoring, refit=False, return_train_score=True, cv = 5, verbose = 2, n_jobs=-1)

    clf.fit(X, y)
    results = clf.cv_results_
    pd.DataFrame(results).to_csv(f"KNN_{w}.csv")
    display(pd.DataFrame(results))



def XGB(X_train, X_valid, y_train, y_valid):
    Clf = make_pipeline(
      MinMaxScaler(),
      XGBClassifier(max_depth = 30, min_child_weight = 3)
    )
    Clf.fit(X_train, y_train)
    trainScore = Clf.score(X_train, y_train) 
    validationScore = Clf.score(X_valid, y_valid)
    print(f"Train Accuracy       : {trainScore * 100:.2f}%")
    print(f"Test Accuracy        : {validationScore * 100:.2f}%")
    predictY = Clf.predict(X_valid) 
    display(pd.DataFrame(classification_report(predictY, y_valid, output_dict=True)))

def RF(X_train, X_valid, y_train, y_valid):
    Clf = make_pipeline(
      MinMaxScaler(),
      RandomForestClassifier(n_estimators=100, max_depth=50)
    )
    Clf.fit(X_train, y_train)
    trainScore = Clf.score(X_train, y_train) 
    validationScore = Clf.score(X_valid, y_valid)
    predictY = Clf.predict(X_valid) 
    print(f"Train Accuracy       : {trainScore * 100:.2f}%")
    print(f"Test Accuracy        : {validationScore * 100:.2f}%")
    display(pd.DataFrame(classification_report(predictY, y_valid, output_dict=True)))

def KNNClassifier(X_train, X_valid, y_train, y_valid):
    Clf = make_pipeline(
      MinMaxScaler(),
      KNeighborsClassifier(n_neighbors = 3 , weights = 'uniform' )
    )
    Clf.fit(X_train, y_train)
    trainScore = Clf.score(X_train, y_train) 
    validationScore = Clf.score(X_valid, y_valid)
    predictY = Clf.predict(X_valid) 
    print(f"Train Accuracy       : {trainScore * 100:.2f}%")
    print(f"Test Accuracy        : {validationScore * 100:.2f}%")
    display(pd.DataFrame(classification_report(predictY, y_valid, output_dict=True)))



In [15]:
def XGBAutomator(X, y):
    mcw = 1
    GridXGB(X, y, mcw)
    
    mcw = 3
    GridXGB(X, y, mcw)
    
    mcw = 5
    GridXGB(X, y, mcw)
    

def RFAutomator(X, y):
    # RF
    mdList = [None, 15, 25, 35, 50]
    for md in mdList:
        GridRF(X, y, md)

def KNNAutomator(X, y):
    # GridKNN
    w = 'uniform'
    GridKNN(X,y,w)

    w = 'distance'
    GridKNN(X,y,w)


def main():
    data = DataImportAndProcessing()
    
    dataNot3 = data[data.outcome != 3].reset_index(drop=True)
    data3 = data[data.outcome == 3].reset_index(drop=True)
  
    X_dataNot3 = dataNot3.drop(columns=['outcome']).to_numpy()
    y_dataNot3 = dataNot3['outcome'].to_numpy()

    X_data3 = data3.drop(columns=['outcome']).to_numpy()
    y_data3 = data3['outcome'].to_numpy()

    X_train_N3, X_valid_N3, y_train_N3, y_valid_N3 = train_test_split(X_dataNot3, y_dataNot3, test_size=0.40)

    X_train_3, X_valid_3, y_train_3, y_valid_3 = train_test_split(X_data3, y_data3, test_size=0.1)
    
    X_train_3 = pd.DataFrame(X_train_3)
    X_train_N3 = pd.DataFrame(X_train_N3)
    X_train = X_train_3.append(X_train_N3)#.to_numpy()
   
    X_valid_3 = pd.DataFrame(X_valid_3)
    X_valid_N3 = pd.DataFrame(X_valid_N3)
    X_valid = X_valid_3.append(X_valid_N3)#.to_numpy()
    

    y_train_3 = pd.DataFrame(y_train_3)
    y_train_N3 = pd.DataFrame(y_train_N3)
    y_train = y_train_3.append(y_train_N3).values.ravel()
    

    y_valid_3 = pd.DataFrame(y_valid_3)
    y_valid_N3 = pd.DataFrame(y_valid_N3)
    y_valid = y_valid_3.append(y_valid_N3).values.ravel()

    ## grid Search
    gridsearch = input("do you want to test grid search? y/n")
    if gridsearch == 'y':
        RFAutomator(X_train, y_train)
        KNNAutomator(X_train, y_train)
        XGBAutomator(X_train, y_train)

    ## best models 

    RF(X_train, X_valid, y_train, y_valid)
    
    XGB(X_train, X_valid, y_train, y_valid)
    
    KNNClassifier(X_train, X_valid, y_train, y_valid)

    
main()

length of filled data:  557340
do you want to test grid search? y/ny
Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  23 out of  25 | elapsed:    7.6s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    8.1s finished


mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0       0.447424      0.029977         0.021183        0.001465   
1       0.752564      0.038099         0.030475        0.007352   
2       1.069092      0.064393         0.042392        0.009074   
3       1.592590      0.054523         0.051006        0.010669   
4       1.845426      0.236269         0.041752        0.009192   

  param_n_estimators                 params  split0_test_recall  \
0                 50   {'n_estimators': 50}            0.581028   
1                 75   {'n_estimators': 75}            0.575593   
2                100  {'n_estimators': 100}            0.575593   
3                150  {'n_estimators': 150}            0.586462   
4                200  {'n_estimators': 200}            0.581028   

   split1_test_recall  split2_test_recall  split3_test_recall  ...  \
0            0.580556            0.592573            0.492512  ...   
1            0.597475            0.581209            0.492512  ...   
2            0.580556            0.569846            0.498068  ...   
3            0.597475            0.569846            0.498068  ...   
4            0.603030            0.569846            0.498068  ...   

   mean_test_Accuracy  std_test_Accuracy  rank_test_Accuracy  \
0            0.807231           0.041609                   3   
1            0.803884           0.041763                   5   
2            0.805579           0.036305                   4   
3            0.808857           0.040232                   2   
4            0.810523           0.040686                   1   

   split0_train_Accuracy  split1_train_Accuracy  split2_train_Accuracy  \
0               0.975052               0.981289               0.979253   
1               0.977131               0.981289               0.977178   
2               0.977131               0.981289               0.979253   
3               0.977131               0.981289               0.979253   
4               0.977131               0.981289               0.979253   

   split3_train_Accuracy  split4_train_Accuracy  mean_train_Accuracy  \
0               0.979253               0.983402             0.979650   
1               0.981328               0.983402             0.980066   
2               0.981328               0.983402             0.980481   
3               0.981328               0.983402             0.980481   
4               0.981328               0.983402             0.980481   

   std_train_Accuracy  
0            0.002765  
1            0.002497  
2            0.002128  
3            0.002128  
4            0.002128  

[5 rows x 51 columns]

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 5 candidates, totalling 25 fits


...

Train Accuracy       : 97.51%
Test Accuracy        : 82.91%


0          1           2    3  accuracy   macro avg  \
precision    0.977901   0.579710    0.768707  0.0  0.829146    0.581580   
recall       0.988827   0.533333    0.801418  0.0  0.829146    0.580895   
f1-score     0.983333   0.555556    0.784722  0.0  0.829146    0.580903   
support    179.000000  75.000000  141.000000  3.0  0.829146  398.000000   

           weighted avg  
precision      0.821382  
recall         0.829146  
f1-score       0.824948  
support      398.000000

Train Accuracy       : 87.87%
Test Accuracy        : 80.90%


D:\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0          1           2    3  accuracy   macro avg  \
precision    0.977901   0.492754    0.755102  0.0  0.809045    0.556439   
recall       0.994382   0.459459    0.760274  0.0  0.809045    0.553529   
f1-score     0.986072   0.475524    0.757679  0.0  0.809045    0.554819   
support    178.000000  74.000000  146.000000  0.0  0.809045  398.000000   

           weighted avg  
precision      0.805967  
recall         0.809045  
f1-score       0.807364  
support      398.000000